In [1]:
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from IPython import embed

In [56]:
class TripletLoss(nn.Module):
    def __init__(self, margin=0.3):
        super().__init__()
        self.margin=margin
        self.ranking_loss=nn.MarginRankingLoss(margin=margin)#计算三元组损失使用的函数
        
    def forward(self, inputs, targets):
        n=inputs.size(0)
        """
        计算图片之间的欧氏距离
        矩阵A,B欧氏距离等于√(A^2 + (B^T)^2 - 2A(B^T))
        """
        #计算A^2
        distance=torch.pow(inputs,2).sum(dim=1, keepdim=True).expand(n,n)
        #计算A^2 + (B^T)^2
        distance=distance+distance.t()
        #计算A^2 + (B^T)^2 - 2A(B^T)
        distance.addmm(1,-2,inputs,inputs.t())
        #计算√(A^2 + (B^T)^2 - 2A(B^T))
        distance=distance.clamp(min=1e-12).sqrt()#该distance矩阵为对称矩阵
        
        print(distance)
        
        #获取对角线
        mask=target.expand(n,n)==target.expand(n,n).t()#mask矩阵用于区分红绿色区域，即正样本区与负样本区，便于进行损失计算。
        
        distance_ap,distance_an=[],[]
        for i in range(n):
            print([mask[i]==0])
            distance_ap.append(distance[i][mask[i]].max().unsqueeze(0))#distance[i][mask[i]]使distance保留正样本区
            distance_an.append(distance[i][mask[i]==0].min().unsqueeze(0))#distance[i][mask[i]==0]使distance保留负样本区
        #经过for循环后，正样本最大距离与负样本最小距离都存储在list当中，需要将list元素连接成一个torch张量
        distance_ap=torch.cat(distance_ap)
        distance_an=torch.cat(distance_an)
        print(distance_an.size())
        y=torch.ones_like(distance_an)
        loss=self.ranking_loss(distance_an, distance_ap, y)
        return loss

In [57]:
if __name__=='__main__':
    target=[1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4,5,5,5,5,6,6,6,6,7,7,7,7,8,8,8,8]
    target=torch.Tensor(target)
    features=torch.rand(32,2048)
    a=TripletLoss()
    loss=a.forward(features,target)
    print(loss)

tensor([[36.9233, 36.8600, 37.0552,  ..., 36.8837, 37.3431, 37.0546],
        [36.8600, 36.7966, 36.9921,  ..., 36.8203, 37.2805, 36.9916],
        [37.0552, 36.9921, 37.1866,  ..., 37.0157, 37.4735, 37.1861],
        ...,
        [36.8837, 36.8203, 37.0157,  ..., 36.8440, 37.3039, 37.0152],
        [37.3431, 37.2805, 37.4735,  ..., 37.3039, 37.7582, 37.4730],
        [37.0546, 36.9916, 37.1861,  ..., 37.0152, 37.4730, 37.1855]])
[tensor([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)]
[tensor([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)]
[tensor([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)]
[tensor([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)]
[tensor([1, 1, 1, 1, 0, 0, 0, 

In [51]:
target=[1,1,0,0]
target=torch.Tensor(target)
print(target==0)

tensor([0, 0, 1, 1], dtype=torch.uint8)
